In [5]:
import numpy as np
import math
import os
import cv2
from PIL import Image, ImageDraw
import datetime

In [2]:
%load_ext autoreload
%autoreload 2
from utils.PathModel import PathModel
from utils.img import imshow
from utils.transform import *
from utils.postprocess import *

In [4]:
img = cv2.imread('data/pics/0.png')
# res = draw_lane(img, model)

In [3]:
a = np.fromfile('data/results/res0.raw',dtype=np.float32)
a = np.array(a)
print(a.shape)
model = PathModel(a)

(1140,)


In [ ]:
import cv2
import numpy as np

MODEL_PATH_MAX_VERTICES_CNT=98

intrinsic = np.array([
  [910., 0., 582.],
  [0., 910., 437.],
  [0.,   0.,   1.]
])

# eon_focal_length=910
# medmodel_zoom = 1.
# MEDMODEL_INPUT_SIZE = (512, 256)
# MEDMODEL_YUV_SIZE = (MEDMODEL_INPUT_SIZE[0], MEDMODEL_INPUT_SIZE[1] * 3 // 2)
# MEDMODEL_CY = 47.6
# intrinsic = np.array(
#   [[ eon_focal_length / medmodel_zoom,    0. ,  0.5 * MEDMODEL_INPUT_SIZE[0]],
#    [   0. ,  eon_focal_length / medmodel_zoom,  MEDMODEL_CY],
#    [   0. ,                            0. ,   1.]])

# BIGMODEL_INPUT_SIZE = (864, 288)
# BIGMODEL_YUV_SIZE = (BIGMODEL_INPUT_SIZE[0], BIGMODEL_INPUT_SIZE[1] * 3 // 2)

# bigmodel_zoom = 1.
# intrinsic = np.array(
#   [[ eon_focal_length / bigmodel_zoom,    0. , 0.5 * BIGMODEL_INPUT_SIZE[0]],
#    [   0. ,  eon_focal_length / bigmodel_zoom,  0.2 * BIGMODEL_INPUT_SIZE[1]],
#    [   0. ,                            0. ,   1.]])

def car_space_to_full_frame(car_space_projective):
	extrinsic = np.array([[ 9.86890774e-03, -9.99951124e-01, -6.39820937e-04, 0.00000000e+00],
	   [-6.46961704e-02, -5.42101086e-20, -9.97905016e-01, 1.22000003e+00],
	   [ 9.97856200e-01,  9.88962594e-03, -6.46930113e-02, 0.00000000e+00]])
# 	extrinsic = np.array([[ 0.  , -1.  ,  0.  ,  0.  ],
#        [ 0.  ,  0.  , -1.  ,  1.22],
#        [ 1.  ,  0.  ,  0.  ,  0.  ]])

	ep = extrinsic.dot(car_space_projective)
	# print(ep.shape)
	kep = intrinsic.dot(ep)
	p_image = np.array([kep[0]/kep[2], kep[1]/kep[2]])
	return p_image

class Pvd():
	class pt():
		def __init__(self,x,y):
			self.x = x
			self.y = y
		def __repr__(self):
			return '('+str(self.x)+','+str(self.y)+')'
	def __init__(self):
		self.cnt = 0
		self.v = []
	def add_pt(self,x,y):
		p = self.pt(x,y)
		self.v.append(p)
	def __repr__(self):
		return str(self.v)

def update_lane_line_data(points, off, is_ghost):
	pvd = Pvd()
	for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2):
		px = float(i)
		py = points[i] - off
		p_car_space = np.array([px, py, 0, 1])
		p_full_frame = car_space_to_full_frame(p_car_space)
		x = p_full_frame[0]
		y = p_full_frame[1]
# 		print(x,y)
		if x<0 or y<0:
			continue
		pvd.add_pt(x,y)
		pvd.cnt += 1
	for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2, 0, -1):
		px = float(i)
		if is_ghost:
			py = points[i]-off
		else:
			py = points[i]+off
		p_car_space = np.array([px, py, 0, 1])
		p_full_frame = car_space_to_full_frame(p_car_space)
		x = p_full_frame[0]
		y = p_full_frame[1]
		if x<0 or y<0:
			continue
		pvd.add_pt(x,y)
		pvd.cnt += 1
	return pvd

def update(p):
	u1 = update_lane_line_data(p.points,0.025*p.prob, False)
	var = min(p.std, 0.7)
	u2 = update_lane_line_data(p.points,-var, True)
	u3 = update_lane_line_data(p.points,var, True)
	return u1,u2,u3

def draw_lane(img, model):
	p1,p2,p3 = update(model.path)
	lp1,lp2,lp3 = update(model.left_lane)
	rp1,rp2,rp3 = update(model.right_lane)
	img = draw(img, p1, (255,0,0))
	img = draw(img, p2, (255,0,0))
	img = draw(img, p3, (255,0,0))
	img = draw(img, lp1, (0,255,0))
	img = draw(img, lp2, (0,255,0))
	img = draw(img, lp3, (0,255,0))
	img = draw(img, rp1, (0,0,255))
	img = draw(img, rp2, (0,0,255))
	img = draw(img, rp3, (0,0,255))

	return img

def draw(img, line, c):
	l = line.v[1:]
	# print(l)
	for j in range(1,len(l)):
		pt1 = (int(l[j-1].x),int(l[j-1].y))
		pt2 = (int(l[j].x),int(l[j].y))
		cv2.line(img, pt1, pt2, c, 4)
	return img

---

In [6]:
pics2 = []
n = datetime.datetime.now()
for i in range(50):
    ii = str(i)
    a = np.fromfile('data/results/ress'+ii+'.raw',dtype=np.float32)
    a = np.array(a)
    model = PathModel(a)
    img = cv2.imread('data/pics/'+ii+'.png')
    res = draw_lane(img.copy(),model)
    pics2.append(res)
print(datetime.datetime.now()-n)

0:00:07.972831


In [8]:
8/50

0.16

In [ ]:
def update_lane_line_data(points, off, is_ghost):
# 	print(points)
# 	print(off)
	pvd = Pvd()
	for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2):
		px = float(i)
		py = points[i] - off
		p_car_space = np.array([px, py, 0, 1])
		p_full_frame = car_space_to_full_frame(p_car_space)
		x = p_full_frame[0]
		y = p_full_frame[1]
#         print(x,y)
		if x<0 or y<0:
			continue
		pvd.add_pt(x,y)
		pvd.cnt += 1
	for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2, 0, -1):
		px = float(i)
		if is_ghost:
			py = points[i]-off
		else:
			py = points[i]+off
		p_car_space = np.array([px, py, 0, 1])
		p_full_frame = car_space_to_full_frame(p_car_space)
		x = p_full_frame[0]
		y = p_full_frame[1]
		if x<0 or y<0:
			continue
		pvd.add_pt(x,y)
		pvd.cnt += 1
	return pvd

In [ ]:
MODEL_PATH_MAX_VERTICES_CNT=98
# rgb_height = 1748/2
# rgb_width = 2328/2

# intrinsic_matrix = np.array([
#   [910., 0., 582.],
#   [0., 910., 437.],
#   [0.,   0.,   1.]
# ])
eon_focal_length=910
medmodel_zoom = 1.
MEDMODEL_INPUT_SIZE = (512, 256)
MEDMODEL_YUV_SIZE = (MEDMODEL_INPUT_SIZE[0], MEDMODEL_INPUT_SIZE[1] * 3 // 2)
MEDMODEL_CY = 47.6
intrinsics = np.array(
  [[ eon_focal_length / medmodel_zoom,    0. ,  0.5 * MEDMODEL_INPUT_SIZE[0]],
   [   0. ,  eon_focal_length / medmodel_zoom,  MEDMODEL_CY],
   [   0. ,                            0. ,   1.]])

def car_space_to_full_frame(car_space_projective):
    extrinsic = np.array([[ 9.86890774e-03, -9.99951124e-01, -6.39820937e-04, 0.00000000e+00],
       [-6.46961704e-02, -5.42101086e-20, -9.97905016e-01, 1.22000003e+00],
       [ 9.97856200e-01,  9.88962594e-03, -6.46930113e-02, 0.00000000e+00]])
    ep = extrinsic.dot(car_space_projective)
    # print(ep.shape)
    kep = intrinsic_matrix.dot(ep)
    p_image = np.array([kep[0]/kep[2], kep[1]/kep[2], 1])
    return p_image

def update_lane_line_data(points, off, is_ghost):
    pvd = {
        'cnt':0,
        'v':[]
    }
    for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2):
        px = float(i)
        py = points[i] - off
        p_car_space = np.array([px, py, 0, 1])
        p_full_frame = car_space_to_full_frame(p_car_space)
        temp = {
            'x':p_full_frame[0],
            'y':p_full_frame[1],
        }
        if temp['x']<0 or temp['y']<0:
            continue
        # if not (px >= 0 and px <= rgb_width and py >= 0 and py <= rgb_height):
            # continue
        pvd['v'].append(temp)
        pvd['cnt'] += 1
    for i in range(MODEL_PATH_MAX_VERTICES_CNT // 2, 0, -1):
        px = float(i)
        if is_ghost:
            py = points[i]-off
        else:
            py = points[i]+off
        p_car_space = np.array([px, py, 0, 1])
        p_full_frame = car_space_to_full_frame(p_car_space)
        temp = {
            'x':p_full_frame[0],
            'y':p_full_frame[1],
        }
        if temp['x']<0 or temp['y']<0:
            continue

        pvd['v'].append(temp)
        pvd['cnt'] += 1
    return pvd

def update(p):
    p1 = update_lane_line_data(p['points'],0.025*p['prob'], False)
    var = min(p['std'], 0.7)
    p2 = update_lane_line_data(p['points'],-var, True)
    p3 = update_lane_line_data(p['points'],var, True)
    return p1,p2,p3

In [ ]:
def draw(line,c):
    l = line['v'][1:]
    for j in range(1,len(l)):
        pt1 = (int(l[j-1]['x']),int(l[j-1]['y']))
        pt2 = (int(l[j]['x']),int(l[j]['y']))
        cv2.line(img, pt1, pt2, c, 4)

In [ ]:
def draw(line,c):
    height = 894
    width = 1164
    img2 = np.array([[[0,0,0]]*width]*height).astype(np.uint8).copy()
    l = line['v'][1:]
    for j in range(1,len(l)):
        pt1 = (int(l[j-1]['x']),int(l[j-1]['y']))
        pt2 = (int(l[j]['x']),int(l[j]['y']))
#         print(pt1,pt2)
        cv2.line(img2, pt1, pt2, c, 10)
        
    return img2

In [ ]:
# extrinsic = get_view_frame_from_road_frame(0.21, -0.0, 0.23, 1.22)
# extrinsic = get_view_frame_from_road_frame(0.25, -0.0, 0.13, 1.55)
# extrinsic = get_view_frame_from_road_frame(0,0,0,1.2)
extrinsic = np.array([[ 9.86890774e-03, -9.99951124e-01, -6.39820937e-04, 0.00000000e+00],
       [-6.46961704e-02, -5.42101086e-20, -9.97905016e-01, 1.22000003e+00],
       [ 9.97856200e-01,  9.88962594e-03, -6.46930113e-02, 0.00000000e+00]])
p1,p2,p3 = update(model.path)
lp1,lp2,lp3 = update(model.left_lane)
rp1,rp2,rp3 = update(model.right_lane)
# img = cv2.imread('data/29/preview.png')
h = 700
img = cv2.resize(img,(512,256))

# draw(lp1,(255,0,0))
# draw(rp1,(0,0,255))
# imshow(img)

In [ ]:
img.shape[:-1]

In [ ]:
l1 = draw(lp1,(1,1,1))
l2 = draw(lp2,(1,1,1))
l3 = draw(lp3,(1,1,1))
left = cv2.resize((l1+l2+l3),(img.shape[1],img.shape[0]))
ll1 = draw(lp1,(255,0,0))
ll2 = draw(lp2,(255,0,0))
ll3 = draw(lp3,(255,0,0))
leftt = cv2.resize((ll1+ll2+ll3),(img.shape[1],img.shape[0]))
r1 = draw(rp1,(1,1,1))
r2 = draw(rp2,(1,1,1))
r3 = draw(rp3,(1,1,1))
right = cv2.resize((r1+r2+r3),(img.shape[1],img.shape[0]))
rr1 = draw(rp1,(0,0,255))
rr2 = draw(rp2,(0,0,255))
rr3 = draw(rp3,(0,0,255))
rightt = cv2.resize((rr1+rr2+rr3),(img.shape[1],img.shape[0]))
imshow(img-(left+right)*img+(leftt+rightt))

In [ ]:
imshow(cv2.imread('data/pics/0.png'))

In [ ]:
import time
from IPython.display import clear_output
pngs = []
for i in range(50):
#     clear_output()
    pngs.append(cv2.imread('data/pics/'+str(i)+'.png'))
#     imshow(a)
#     time.sleep(0.1)

In [ ]:
for i in range(50):
    clear_output()
    imshow(pics[i])
#     time.sleep(0.1)

In [ ]:
from PIL import Image, ImageDraw
pics = []
for i in range(50):
    pics.append(Image.open('data/pics/'+str(i)+'.png'))

In [ ]:
from IPython.display import Image, display
X = Image(url='test.gif')
display(X)

In [1]:
from IPython.display import Image, display
X = Image(url='test2.gif')
display(X)

In [9]:
from PIL import Image, ImageDraw
imgs2 = [Image.fromarray(cv2.cvtColor(cv2.resize(i,(i.shape[1]//2,i.shape[0]//2)),cv2.COLOR_BGR2RGB)) for i in pics2]
imgs2[0].save('test2.gif', format='GIF', append_images=imgs2[1:], save_all=True, duration=70, loop=0)

In [ ]:
imgs2[0].save('test2.gif', format='GIF', append_images=imgs2[1:], save_all=True, duration=100, loop=0)